In [4]:
import pandas as pd

def get_kcd_chapter(code):
    """
    KCD 코드(문자열)를 받아서 KCD-10 대분류명(장)을 반환합니다.
    사용자가 첨부한 이미지의 분류 체계를 따릅니다.
    """
    if not code or pd.isna(code):
        return "분류 불명"

    # 'J03', 'K29'와 같은 코드에서 첫 글자를 추출
    prefix = code[0].upper()

    try:
        # 'C', 'D', 'H' 코드의 숫자 부분을 확인하기 위함
        num_part = int(code[1:3])
    except (ValueError, TypeError):
        num_part = -1 # 숫자로 변환 안되는 코드(e.g. 'S') 대비

    # KCD-10 (ICD-10) '장(Chapter)' 분류 기준 매핑
    if prefix in ['A', 'B']:
        return "01. 감염성 및 기생충성 질환 (A00-B99)"
    
    if prefix == 'C':
        return "02. 신생물 (C00-D48)"
    
    if prefix == 'D':
        if 0 <= num_part <= 48:
            return "02. 신생물 (C00-D48)"
        elif 50 <= num_part <= 89:
            return "03. 혈액/조혈기관/면역 (D50-D89)"
    
    if prefix == 'E':
        return "04. 내분비, 영양 및 대사 질환 (E00-E90)"
    
    if prefix == 'F':
        return "05. 정신 및 행동 장애 (F00-F99)"
    
    if prefix == 'G':
        return "06. 신경계통의 질환 (G00-G99)"
    
    if prefix == 'H':
        if 0 <= num_part <= 59:
            return "07. 눈 및 눈 부속기 질환 (H00-H59)"
        elif 60 <= num_part <= 95:
            return "08. 귀 및 유돌 질환 (H60-H95)"
    
    if prefix == 'I':
        return "09. 순환기계통의 질환 (I00-I99)"
    
    if prefix == 'J':
        return "10. 호흡기계통의 질환 (J00-J99)"
    
    if prefix == 'K':
        return "11. 소화기계통의 질환 (K00-K93)"
    
    if prefix == 'L':
        return "12. 피부 및 피하조직의 질환 (L00-L99)"
    
    if prefix == 'M':
        return "13. 근골격계통 및 결합조직 질환 (M00-M99)"
    
    if prefix == 'N':
        return "14. 비뇨생식계통의 질환 (N00-N99)"
    
    if prefix == 'O':
        return "15. 임신, 출산 및 산후기 (O00-O99)"
    
    if prefix == 'P':
        return "16. 출생전후기 기원 특정 병태 (P00-P96)"
    
    if prefix == 'Q':
        return "17. 선천기형, 변형 및 염색체 이상 (Q00-Q99)"
    
    if prefix == 'R':
        return "18. 증상, 징후, 임상/검사 이상 (R00-R99)"
    
    if prefix in ['S', 'T']:
        return "19. 손상, 중독 및 외인 (S00-T98)"
    
    if prefix in ['V', 'W', 'X', 'Y']:
        return "20. 질병이환 및 사망의 외인 (V01-Y98)"
    
    if prefix == 'Z':
        return "21. 건강상태 영향요인 (Z00-Z99)"
    
    if prefix == 'U':
        return "22. 특수목적 코드 (U00-U85)"

    return f"분류 불명 ({code})"

# --- 메인 스크립트 로직 ---

def summarize_kcd_data_detailed(file_path='/home/hashjamm/project_data/disease_network/target_diseases_info.csv'):
    """
    CSV 파일을 읽어 KCD-10 대분류 기준으로 상세 집계하고 
    넘버링 순서대로 정렬하여 출력 및 CSV 파일로 저장합니다.
    """
    try:
        # 1. CSV 파일 로드
        df = pd.read_csv(file_path)
        
        # 2. 'code' 열을 기준으로 KCD-10 대분류 라벨링 적용
        #    'code'가 비어있을(NaN) 경우를 대비해 .astype(str) 추가
        df['kcd_chapter'] = df['code'].astype(str).apply(get_kcd_chapter)
        
        # 3. KCD 대분류(kcd_chapter) 기준으로 상세 집계 (Aggregation)
        #    (사용자 요청사항)
        summary = df.groupby('kcd_chapter').agg(
            code_count=('code', 'nunique'),  # 1. 포함된 고유 코드의 개수
            count_sum=('count', 'sum'),      # 2. count의 합산
            count_mean=('count', 'mean')     # 3. count의 평균
        )
        
        # 4. 'kcd_chapter'의 넘버링 순서대로 정렬 (인덱스 기준 정렬)
        summary = summary.sort_index()
        
        # 5. 'percent' 재계산 (count 합산을 기반으로) (사용자 요청사항 4)
        total_count_sum = summary['count_sum'].sum()
        summary['percent_new'] = (summary['count_sum'] / total_count_sum)
        
        # 6. 결과 포맷팅 (출력용)
        summary['count_mean'] = summary['count_mean'].round(2)
        summary['percent_formatted'] = (summary['percent_new'] * 100).map('{:,.2f}%'.format)
        
        # 7. 최종 저장할 DataFrame 선택 (포맷팅 안된 원본 percent_new 포함)
        #    CSV로 저장할 때는 포맷팅된 문자열('%')보다 숫자(percent_new)가 유용합니다.
        csv_summary = summary[['code_count', 'count_sum', 'count_mean', 'percent_new']]
        
        # 8. 화면(터미널)에 출력할 DataFrame (포맷팅된 percent_formatted 포함)
        print_summary = summary[['code_count', 'count_sum', 'count_mean', 'percent_formatted']]
        
        print(f"--- KCD-10 대분류 기준 상세 집계 (파일: {file_path}) ---")
        print(print_summary)
        
        print("\n--- 총계 ---")
        print(f"총 고유 코드 수: {csv_summary['code_count'].sum():,}")
        print(f"총 Count 합계: {csv_summary['count_sum'].sum():,}")
        print(f"총 Percent 합계: {csv_summary['percent_new'].sum() * 100:.2f}%")

        # 9. 집계 결과를 CSV 파일로 저장
        output_filename = 'kcd_summary_sorted.csv'
        csv_summary.to_csv(output_filename, encoding='utf-8-sig') # 포맷팅 안된 'percent_new'가 저장됨
        print(f"\n성공: 집계 결과가 '{output_filename}' 파일로 저장되었습니다.")
        print(f"(저장된 percent 열은 0.2913... 와 같은 숫자 형식입니다.)")


    except FileNotFoundError:
        print(f"오류: '{file_path}' 파일을 찾을 수 없습니다.")
        print("스크립트와 동일한 디렉토리에 파일이 있는지 확인하세요.")
    except KeyError:
        print(f"오류: CSV 파일에 'code' 또는 'count' 열이 없습니다.")
        if 'df' in locals():
            print(f"파일에서 찾은 열: {df.columns.to_list()}")
    except Exception as e:
        print(f"알 수 없는 오류가 발생했습니다: {e}")

# 스크립트 실행
if __name__ == "__main__":
    summarize_kcd_data_detailed()

--- KCD-10 대분류 기준 상세 집계 (파일: /home/hashjamm/project_data/disease_network/target_diseases_info.csv) ---
                                 code_count  count_sum  count_mean  \
kcd_chapter                                                          
01. 감염성 및 기생충성 질환 (A00-B99)              83     141565     1705.60   
02. 신생물 (C00-D48)                       102      22657      222.13   
03. 혈액/조혈기관/면역 (D50-D89)                 15       5796      386.40   
04. 내분비, 영양 및 대사 질환 (E00-E90)            42      28840      686.67   
05. 정신 및 행동 장애 (F00-F99)                 53      21598      407.51   
06. 신경계통의 질환 (G00-G99)                   54      28827      533.83   
07. 눈 및 눈 부속기 질환 (H00-H59)               44     179378     4076.77   
08. 귀 및 유돌 질환 (H60-H95)                  21      78365     3731.67   
09. 순환기계통의 질환 (I00-I99)                  63      56582      898.13   
10. 호흡기계통의 질환 (J00-J99)                  54     690309    12783.50   
11. 소화기계통의 질환 (K00-K93)                  70     228873   